In [1]:
#from google.colab import files
#uploaded = files.upload()

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [3]:
churnData = pd.read_csv('files_for_lab/customer_churn.csv')
churnData.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'files_for_lab/customer_churn.csv'

In [ ]:
churnData.info()

In [ ]:
# drop empty rows in total charges, converts to float and list
display(churnData['TotalCharges'].value_counts())
rows_to_drop = list(churnData[churnData['TotalCharges'] == ' '].index)
churnData.drop(churnData.index[rows_to_drop],inplace=True)
display(churnData['TotalCharges'].value_counts())
churnData['TotalCharges'] = list(map(float,churnData['TotalCharges']))

In [ ]:
churnData.info()

In [ ]:
churnData['Churn'].value_counts()

As you can see there is a huge imbalance in the representation of the two categories 

In [ ]:
churnData.drop(columns="customerID",inplace=True)

In [ ]:
# select categorical
cat_cols = churnData.select_dtypes(include='object')
cat_cols.head()

One hot enconding

In [ ]:
churnData['gender'] = np.where(churnData['gender'] == 'Male',1,0)

my_cols = ['Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection',
           'TechSupport','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']

for col in my_cols:
  churnData[col] = np.where(churnData[col] == 'Yes',1,0)

churnData.head()

In [ ]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
cat.head()

In [ ]:
# lab is above manual conversion of categorical, num is numerical columns
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

In [ ]:
y = pd.DataFrame(data=churnData, columns=['Churn'])
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [ ]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

With this imbalance, when we tried to fit the model it still gave us an accuracy of 80%

In [ ]:
y.value_counts()

Note: Even if we just blindly say that each prediction is **No**, we would still get an accuracy of 

In [ ]:
print("The accuracy of a blind guess is: %4.2f " % (5163/(5163+1869)))

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

# Lets increase the imbalance and see how the model works 

In [ ]:
data = pd.concat([X,y], axis=1, sort=False)
data.head()

In [ ]:
yes = data[data['Churn']=='Yes']
no = data[data['Churn']=='No']
yes = yes.sample(500)

In [ ]:
data = pd.concat([yes,no], axis=0)
print(data['Churn'].value_counts())
data.head()

In [ ]:
#shuffling the data
data = data.sample(frac=1)
data['Churn'].value_counts()

In [ ]:
X = data.drop('Churn', axis=1)
X.head()

In [ ]:
y = data['Churn']
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [ ]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

Despite the advantage of balancing classes, these techniques also have their weaknesses (there is no free lunch). The simplest implementation of over-sampling is to duplicate random records from the minority class, which can cause overfitting. In under-sampling, the simplest technique involves removing random records from the majority class, which can cause loss of information.

In [ ]:
counts = churnData['Churn'].value_counts()
yes = churnData[churnData['Churn']=='Yes'].sample(counts[0], replace=True)
no = churnData[churnData['Churn']=='No']
data = pd.concat([yes,no], axis=0)
data = data.sample(frac=1)
data['Churn'].value_counts()

In contrast, before we had.

In [ ]:
counts = churnData['Churn'].value_counts()
counts

In [ ]:
cat = pd.get_dummies(data[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = data[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = data.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

In [ ]:
transformer = StandardScaler().fit(X)
scaled_x = transformer.transform(X)
y = data['Churn']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=100)

In [ ]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model is: %4.2f "% (classification.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

Other more sophisticated resampling techniques include:

Cluster the records of the majority class, and do the under-sampling by removing records from each cluster, thus seeking to preserve information. 
In over-sampling, instead of creating exact copies of the minority class records, we can introduce small variations into those copies, creating more diverse synthetic samples.

In [ ]:
#pip install imblearn

In [ ]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Downsampling
rus = RandomUnderSampler() 
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_rus, y_rus = rus.fit_sample(X, y)

In [ ]:
y.value_counts()

In [ ]:
y_rus = pd.DataFrame(data=y_rus.flatten())
y_rus.value_counts()

In [ ]:
transformer = StandardScaler().fit(X_rus)
X = transformer.transform(X_rus)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_rus, test_size=0.3, random_state=100)

In [ ]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model after undersampling is: %4.2f "% (classification.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model after undersampling is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

In [ ]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

In [ ]:
ros = RandomOverSampler()
#X = churnData[['tenure', 'SeniorCitizen','MonthlyCharges']]
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_ros, y_ros = ros.fit_sample(X, y)

In [ ]:
y.value_counts()

In [ ]:
y_ros = pd.DataFrame(data=y_ros.flatten())
y_ros.value_counts()

In [ ]:
transformer = StandardScaler().fit(X_ros)
X = transformer.transform(X_ros)

In [ ]:
classification = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = classification.predict(X_test)
print("The accuracy of the logistic_regression model after oversampling is: %4.2f "% (classification.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("The kappa of the logistic regression model after undersampling is: %4.2f " %(cohen_kappa_score(y_pred,y_test)) )

### Synthetic Minority Oversampling TEchnique (SMOTE)

In [ ]:
cat = pd.get_dummies(churnData[['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']], drop_first=True)
lab = churnData[['gender','Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]
num = churnData.select_dtypes(include=['float64','int64'])
X = pd.concat([num,lab,cat], axis=1, sort=False)
X.head()

In [4]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']
X_sm, y_sm = smote.fit_sample(X, y)
y_sm = pd.DataFrame(data=y_sm.flatten())
y_sm.value_counts()

NameError: name 'X' is not defined

### UnderSampling using TomekLinks 

Tomek links are pairs of very close instances, but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [ ]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(X, y)
y_tl = pd.DataFrame(data=y_tl.flatten())
y_tl.value_counts()

In [ ]:
X_tl2, y_tl2 = tl.fit_sample(X_tl, y_tl)
y_tl2 = pd.DataFrame(data=y_tl2.flatten())
y_tl2.value_counts()

In [ ]:
# It does not make the two classes equal but only removes the points from the majority 
# class that are close to other poitns in minority class

In [ ]:
# Similar way it works for multi label classification models 
# But it can be a little tricky 

In [ ]:
transformer = StandardScaler().fit(X)
X = transformer.transform(X)
y = churnData['Churn']

In [ ]:
y[6000:7000] = 'Hello'
y.value_counts()

In [ ]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(X, y)
y_tl = pd.DataFrame(data=y_tl.flatten())
y_tl.value_counts()

In [ ]:
X_sm, y_sm = smote.fit_sample(X, y)
y_sm = pd.DataFrame(data=y_sm.flatten())
y_sm.value_counts()